In [ ]:
import os
import re
import json
import time
import math
import dill
import pandas as pd

from urllib.request import urlopen
import numpy as np

from sqlalchemy import (
    create_engine, Column, Integer, String, Date, MetaData, event, Table, text,
    LargeBinary,  BigInteger, SmallInteger, ForeignKey, Float, inspect
)
import ast
from sqlalchemy.dialects.postgresql import JSON, JSONB  # if not already imported

from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.engine import Engine
from sqlalchemy.exc import ProgrammingError

# If your project structure matches the original, keep these:
from src.helpers import *
from src.dbutils import *
from src.ORMutils import *
from src.models import *
from src.geo import *
from src.pdfutils import *

# Geo / GIS helpers used later
import fiona
import geopandas as gpd
from shapely import wkb, from_wkt  # Shapely 2.x
from geoalchemy2.shape import from_shape

import pprint

PLUTO_VERSION = "25v2_1"



In [ ]:
with open("environment_data/table_dicts.pkl", "rb") as f:
    env = dill.load(f)
globals().update(env)

* ### Create the database engine that will be used throughout the rest of the notebook.

In [ ]:
import getpass
from urllib.parse import quote_plus

# your Linux login must match a PostgreSQL role with rights to create DBs on first run
PGUSER = "jam"  # or: getpass.getuser()
PGPASSWORD = None  # not used with peer auth
PGHOST = "/run/postgresql"  # socket directory on Arch/EndeavourOS; Debian/RHEL often /var/run/postgresql
PGPORT = "5432"   # ignored for sockets but harmless
PGDATABASE = "nyc_data"

# Force socket usage via query param (URL-encode the path just to be safe)
_socket = quote_plus(PGHOST)
POSTGRES_URL = None  # leave None so we can build URLs per DB below

# def make_url(dbname: str) -> str:
#     return f"postgresql+psycopg2://{PGUSER}@/{dbname}?host={_socket}"

In [ ]:

# admin connection (to create DB if missing)
admin_engine = create_engine(make_url("postgres", PGUSER=PGUSER, _socket=_socket), future=True)
# target DB connection
engine = create_engine(make_url(PGDATABASE, PGUSER=PGUSER, _socket=_socket), future=True)

In [ ]:
import re
from sqlalchemy import text

SAFE_DB_RE = re.compile(r"^[a-z_][a-z0-9_]*$")  # postgres-friendly
if not SAFE_DB_RE.fullmatch(PGDATABASE):
    raise ValueError(f"Unsafe database name: {PGDATABASE!r}")

with admin_engine.connect() as conn:
    exists = conn.execute(
        text("SELECT 1 FROM pg_database WHERE datname = :d"),
        {"d": PGDATABASE},
    ).scalar()

if not exists:
    # autocommit required for CREATE DATABASE
    with admin_engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(text(f"CREATE DATABASE {PGDATABASE}"))


In [ ]:

# Create engine to the target database
engine = create_engine(
    make_url(PGDATABASE, PGUSER=PGUSER, _socket=_socket),
    echo=False,
    future=True,
)

SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False, future=True)

# Ensure PostGIS is available
with engine.begin() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis"))

* #### Load the objects created in previous notebooks

In [ ]:
metadata = MetaData()
Base.metadata.reflect(bind=engine)

* ### Create lookup tables variables identified as categorical and for which definitions were extracted from the metadata in the previous notebook.

* There are borough codes in the PLUTO dataset, but annyoingly, in contrast to most other datasets, the borough code is a two letter inital like "BK" or "BX". Also in the PLUTO dataset, "Sanitation Borough" does use the standard numeric codes that most other NYC OpenData datasets use. All this is is to say that it requires special handling separate from my system to extract categories and create lookup tables for them programatically.

In [ ]:
multicolumns = {'zoning_district': 4, 'commercial_overlay': 2, 'special_purpose_district': 3}

for dataset in dataset_info_dict.values():
    for name, repetitions in multicolumns.items():
        for k in dataset.col_customizations.keys():
            if dataset.col_customizations[k].new_name is None:
                dataset.col_customizations[k].new_name = dataset.col_customizations[k].short_name
        _ = {k: v for k, v in dataset.col_customizations.items()
             if dataset.col_customizations[k].new_name.startswith(name)}
        _ = [v for k, v in dataset.col_customizations.items()
             if dataset.col_customizations[k].new_name.endswith("_1")]

## Import the MaPLUTO data:
* List the layers in the file
* In this case there is only one layer, so it isn't necessary to know and specify which one to import, but including anyway for future reference.

In [ ]:
# pluto_version = "25v2_1"
gdb_path = f"{PROJECT_DATA}/files_to_use/MapPLUTO{PLUTO_VERSION}.gdb"

geodata = {}
layers = fiona.listlayers(gdb_path)
for layer in layers:
    gdf_layer = gpd.read_file(gdb_path, layer=layer)
    try:
        gdf_layer['wkb'] = gdf_layer['geometry'].apply(lambda geom: geom.wkb if geom else None)
    except KeyError:
        pass
    geodata[layer] = gdf_layer

gdf = geodata[f'MapPLUTO_{PLUTO_VERSION}_clipped']

In [ ]:
def is_whole_number_series(s: pd.Series) -> bool:
    try:
        notna = s.notna()
        return (notna & ((s[notna] % 1) == 0)).all()
    except Exception:
        return False

for col in gdf.columns:
    if gdf[col].dtype == float and is_whole_number_series(gdf[col]):
        gdf[col] = gdf[col].astype('Int64')

inspector = inspect(engine)

col_customization_dict = dataset_info_dict['mapPLUTO'].col_customizations
rename_mappings = {v.short_name: v.new_name for v in col_customization_dict.values()}
gdf = gdf.rename(columns=rename_mappings)

more_mappings = {
    "HealthCenterDistrict": "health_center_district",
    "SanitDistrict": "sanitation_district_number",
    "Sanitboro": "sanitation_district_boro",
    "FIRM07_FLAG": "2007_flood_insurance_rate_map_indicator",
    "PFIRM15_FLAG": "2015_preliminary_flood_insurance_rate_map",
}
gdf = gdf.rename(columns=more_mappings)
gdf = add_padding_to_special_columns(gdf, SPECIAL_PADDING_COLS)
gdf.columns = list(map(str.lower, gdf.columns))

# after you load dataset_info_dict and before create_dynamic_table_class() / create_all()
for k, v in col_customization_dict.items():
    if v.new_name == "version_number":
        v.dtype = "String"   # ensure TEXT in Postgres

In [ ]:
def map_custom_dtype(dtype):
    if dtype == 'Integer':
        return Integer
    elif dtype == 'String':
        return String
    elif dtype == 'Float':
        return Float
    elif dtype == 'Date':
        return Date
    elif dtype == 'LargeBinary':
        return LargeBinary
    else:
        raise ValueError(f"Unsupported dtype: {dtype}")

def create_dynamic_table_class(table_name, col_customization_dict):
    # IMPORTANT: Use LargeBinary for 'geometry' because you actually store WKB bytes there.
    table_name = table_name.lower()
    attrs = {
        '__tablename__': table_name,
        'id': Column(Integer, primary_key=True, autoincrement=True),
        'geometry': Column(LargeBinary),  # store WKB bytes (BYTEA)
        'wkb': Column(LargeBinary),       # kept for parity with original schema
        'shape_leng': Column(Float),
        'shape_area': Column(Float),
        '__table_args__': {'extend_existing': True}
    }
    for k, v in col_customization_dict.items():
        if any(name for name in multicolumns if name in k):
            k = re.sub('_[0-9]$', '', k)
        col_type = map_custom_dtype(v.dtype)
        attrs[v.new_name] = Column(col_type)
    return type(table_name, (Base,), attrs)

MapPLUTO_Clipped = create_dynamic_table_class(f'MapPLUTO_{PLUTO_VERSION}_clipped', col_customization_dict)
Base.metadata.create_all(engine)

In [ ]:
gdf.columns
list(map(str.lower, gdf.columns))

In [ ]:
import math, re

def normalize_zip(v):
    if v is None: 
        return None
    # floats/ints -> 5-digit string
    if isinstance(v, (int, float)) and not (isinstance(v, float) and math.isnan(v)):
        return f"{int(v):05d}"
    s = str(v).strip()
    if s == "": 
        return None
    # '10013.0' -> '10013'
    m = re.fullmatch(r"\d+(?:\.0+)?", s)
    if m:
        return f"{int(float(s)):05d}"
    # '10013-1234' or '10013 1234' -> '10013-1234'
    m = re.fullmatch(r"(\d{5})[-\s]?(\d{4})", s)
    if m:
        return f"{m.group(1)}-{m.group(2)}"
    # anything else: leave as-is (rare)
    return s


batch_size = 100000
with SessionLocal() as session:
    for start in range(0, len(gdf), batch_size):
        batch = gdf.iloc[start:start + batch_size]
        for idx, row in batch.iterrows():
            try:
                if 'apportionment_date' in row and row['apportionment_date']:
                    row['apportionment_date'] = parseDateString(row['apportionment_date'])
                for col in gdf.columns:
                    # print(f"Processing column: {col}")
                    val = row[col]
                    # print(f"Value: {val}, Type: {type(val)}")
                    if isinstance(val, pd.Series):
                        try:
                            first_value = val.iloc[0]
                            row[col] = first_value
                        except Exception as e:
                            print(f"Error processing Series in column {col} at row {idx}: {e}")
                    if pd.isna(val):
                        row[col] = None
                geometry_wkb = row['geometry'].wkb if row['geometry'] is not None else None
                # in your insert loop before constructing MapPLUTO_Clipped(...)
                if 'zip_code' in row:
                    row['zip_code'] = normalize_zip(row['zip_code'])
                pluto_entry = MapPLUTO_Clipped(
                    geometry=geometry_wkb,
                    **{col: row[col] for col in gdf.columns if col != 'geometry'}
                )
                session.add(pluto_entry)
            except Exception as e:
                print(f"Error at row index {idx}")
                for col in gdf.columns:
                    try:
                        print(f"Column: {col}, Value: {row[col]}, Type: {type(row[col])}")
                    except Exception as sub_e:
                        print(f"Error printing column {col}: {sub_e}")
                raise e
        session.commit()

In [ ]:
pd.set_option('display.max_columns', None)
print(gdf[gdf.eq("E-61").any(axis=1)])


In [ ]:
from shapely import wkb
import pandas as pd

df = pd.read_sql(f'SELECT zip_code, geometry FROM "mappluto_{PLUTO_VERSION}_clipped"', engine)

def to_geom(val):
    if val is None:
        return None
    # psycopg2 often returns memoryview for bytea
    if isinstance(val, memoryview):
        return wkb.loads(val.tobytes())
    if isinstance(val, (bytes, bytearray)):
        return wkb.loads(bytes(val))
    # if somehow a hex text like "\x010203..." sneaks through:
    if isinstance(val, str) and val.startswith("\\x"):
        return wkb.loads(bytes.fromhex(val[2:]))
    raise TypeError(f"Unexpected geometry value type: {type(val)}")

df["geometry"] = df["geometry"].apply(to_geom)
gdf_map = gpd.GeoDataFrame(df, geometry="geometry")


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import wkb
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import networkx as nx
from sqlalchemy import create_engine, event, text

# Debug: Print the DataFrame columns
print("DataFrame columns:", df.columns)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Print the GeoDataFrame
print(gdf.head())

# Ensure that zip_code is preserved during the dissolve process
merged_gdf = gdf.dissolve(by='zip_code', aggfunc={'zip_code': 'first'})  # Explicit aggregation of zip_code

# Check if zip_code is now present after dissolving
print(merged_gdf.columns)  # Should include 'zip_code'

# Create a new adjacency graph based on the merged geometries
G = nx.Graph()

# Add nodes and edges based on adjacency of merged shapes
for i, shape1 in merged_gdf.iterrows():
    for j, shape2 in merged_gdf.iterrows():
        if i != j and shape1.geometry.touches(shape2.geometry):
            G.add_edge(i, j)

# Perform graph coloring to ensure adjacent shapes don't share the same color
color_map = nx.coloring.greedy_color(G, strategy="largest_first")

# Plot the map with the colors assigned
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Normalize the color map to cover the full range of the node indices
norm = mcolors.Normalize(vmin=min(color_map.values()), vmax=max(color_map.values()))
sm = plt.cm.ScalarMappable(cmap=plt.cm.tab20, norm=norm)

# Color the merged geometries based on the graph coloring using the full palette
merged_gdf['color'] = merged_gdf.index.map(color_map)
merged_gdf.plot(ax=ax, color=[sm.to_rgba(i) for i in merged_gdf['color']], edgecolor='black', linewidth=0, legend=False)

# Add labels at the center of each merged shape
for _, row in merged_gdf.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y, str(row['zip_code']), fontsize=2, ha='center', va='center')

# Add a colorbar to visualize the full range of colors
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Color Range (Graph Coloring)', rotation=270, labelpad=20)

plt.savefig(f"{PROJECT_DATA}/figures/map_output_zip_shuffled2.pdf", format="pdf")

plt.show()

In [ ]:
def _pg_int_bounds(sqlatype):
    # Map SQL types to numpy bounds
    if isinstance(sqlatype, SmallInteger):
        return np.iinfo(np.int16).min, np.iinfo(np.int16).max
    elif isinstance(sqlatype, BigInteger):
        return np.iinfo(np.int64).min, np.iinfo(np.int64).max
    else:
        # Default Integer in Postgres is 32-bit
        return np.iinfo(np.int32).min, np.iinfo(np.int32).max

def enforce_integer_bounds(df, DynamicTable):
    """
    For each INTEGER-like column in DynamicTable, coerce df[col] to numeric and
    replace out-of-range values with None. Logs offenders for inspection.
    """
    for col in df.columns:
        if col not in DynamicTable.__table__.columns:
            continue
        col_type = DynamicTable.__table__.columns[col].type

        # Only care about integer-ish SQL column types
        if not isinstance(col_type, (SmallInteger, Integer, BigInteger)):
            continue

        lo, hi = _pg_int_bounds(col_type)

        # Coerce to numeric safely; keep strings like "7" working
        s = pd.to_numeric(df[col], errors="coerce")  # floats OK here
        # If column is conceptually integer, store as pandas nullable Int64 to preserve NULLs
        # but do comparisons on the coerced float/int series 's'
        mask_oob = (s.notna()) & ((s < lo) | (s > hi))

        if mask_oob.any():
            bad = df.loc[mask_oob, col]
            # Show a small sample for debugging
            sample_vals = bad.head(5).tolist()
            print(
                f"❗ Out-of-range detected in integer column '{col}' "
                f"({len(bad)} rows). Bounds [{lo}, {hi}]. Sample: {sample_vals}"
            )
            # Null them out so insert won't fail
            df.loc[mask_oob, col] = None

        # Final cast to pandas nullable integer if column is nullable; else leave as numeric
        # (Your pipeline already handles nullable detection; this is safe either way.)
        try:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        except Exception:
            # If the column had non-integer strings, keep as object; the DB will cast or fail elsewhere
            pass

from sqlalchemy.exc import DataError

def insert_with_diagnosis(conn, insert_stmt, batch_rows, key_hint='sid'):
    try:
        # conn.execute(insert_stmt, batch_rows)
        insert_with_diagnosis(conn, insert_stmt, batch_rows, key_hint='sid')
        return
    except DataError as e:
        print("❌ Batch insert failed; diagnosing offending rows…", e)
        lo, hi = 0, len(batch_rows)
        # binary search the smallest failing prefix
        while lo + 1 < hi:
            mid = (lo + hi) // 2
            try:
                conn.execute(insert_stmt, batch_rows[:mid])
                lo = mid
            except DataError:
                hi = mid
        bad = batch_rows[lo:hi]  # minimal failing slice
        print(f"🚩 Offending rows count: {len(bad)}")
        if key_hint and key_hint in bad[0]:
            print(f"Example offending {key_hint}: {bad[0][key_hint]}")
        # You can drop or sanitize and retry:
        # conn.execute(insert_stmt, batch_rows[:lo] + batch_rows[hi:])
        raise


In [ ]:
def enforce_integer_bounds(df, DynamicTable):
    """
    For each INTEGER-like column in DynamicTable, coerce df[col] to numeric and
    replace out-of-range values with None. Logs offenders for inspection.
    """
    for col in df.columns:
        if col not in DynamicTable.__table__.columns:
            continue
        col_type = DynamicTable.__table__.columns[col].type

        # Only care about integer-ish SQL column types
        if not isinstance(col_type, (SmallInteger, Integer, BigInteger)):
            continue

        lo, hi = _pg_int_bounds(col_type)

        # Coerce to numeric safely; keep strings like "7" working
        s = pd.to_numeric(df[col], errors="coerce")  # floats OK here
        # If column is conceptually integer, store as pandas nullable Int64 to preserve NULLs
        # but do comparisons on the coerced float/int series 's'
        mask_oob = (s.notna()) & ((s < lo) | (s > hi))

        if mask_oob.any():
            bad = df.loc[mask_oob, col]
            # Show a small sample for debugging
            sample_vals = bad.head(5).tolist()
            print(
                f"❗ Out-of-range detected in integer column '{col}' "
                f"({len(bad)} rows). Bounds [{lo}, {hi}]. Sample: {sample_vals}"
            )
            # Null them out so insert won't fail
            df.loc[mask_oob, col] = None

        # Final cast to pandas nullable integer if column is nullable; else leave as numeric
        # (Your pipeline already handles nullable detection; this is safe either way.)
        try:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        except Exception:
            # If the column had non-integer strings, keep as object; the DB will cast or fail elsewhere
            pass


In [ ]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from src.dbutils import *


In [ ]:


# # Run for all JSON datasets
for name, dataset in dataset_info_dict.items():
    if dataset.format == 'json':
        print(f'Starting dataset {dataset.short_name}')
        insert_dataset(engine, dataset, dataset.dataset_path, dataset.col_types, batch_size=100000)
